**Load the required data and dataset**

In [ ]:
# Load the required datasets 
import pandas as pd 
import panel as pn
import hvplot.pandas
import holoviews as hv
import numpy as np
import seaborn as sns 
import ast
import warnings

warnings.filterwarnings("ignore")

hv.extension('bokeh')

pn.extension('tabulator', sizing_mode="stretch_width")


In [ ]:
# Load the relevant data
portfolio = pd.read_csv("data/output/portfolio_clean.csv")
profile = pd.read_csv("data/output/profile_clean.csv")

# Glimpse of the dataset 
portfolio

### Data Preprocessing

In [ ]:
# Create new age groups 
bins= [18,36,56,76,150]
labels = ['Youth','Middle-Aged','Retired','Aging']


profile['ageGroup'] = pd.cut(profile['age'], bins=bins, labels=labels, right=False)
profile.head()

**Set up the environment**

In [ ]:
def environment():
    try:
        get_ipython()
        return "notebook"
    except:
        return "server"
environment()

**Aesthetics for UI**

In [ ]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
)

In [ ]:
# Make the dataframe interactive 
iportfolio = portfolio.interactive()
iportfolio

In [ ]:
iprofile = profile.interactive()
iprofile.head()

**Define panel widgets**

In [ ]:
# X axis values 
xaxis = pn.widgets.RadioButtonGroup(
    name='X axis', 
    options=['reward', 'difficulty'],
    button_type='success'
)

# Y axis values 
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['difficulty', 'duration'],
    button_type='success'
)

# demographics values 
demographics = pn.widgets.RadioButtonGroup(
    name='Demographics', 
    options=['gender', 'ageGroup'],
    button_type='success'
)


In [ ]:
portfolio_pipeline = (iportfolio) # for portfolio

#for profile
profile_pipeline = (iprofile[[demographics, "income"]].groupby(demographics).mean().reset_index())

In [ ]:
if environment()=="server":
    theme="fast"
else:
    theme="simple"

In [ ]:
# itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10) #for portfolio

profile_table = profile_pipeline.pipe(pn.widgets.Tabulator, pagination='remote')
profile_table

In [ ]:
# ihvplot = ipipeline.hvplot(x='mpg', y=yaxis, by='origin', color=PALETTE, line_width=6, height=400)
# ihvplot

ihvscatter = portfolio_pipeline.hvplot(x=xaxis, y=yaxis, by='offer_type', 
                              color=PALETTE, kind="scatter", height=600)
ihvscatter

In [ ]:
ihvbarchart = profile_pipeline.hvplot(x=demographics, y="income", 
                              color=PALETTE, kind="bar", width=600)
ihvbarchart

In [ ]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive', 
    sidebar=['X axis' , xaxis, 'Y axis' , yaxis, 'Demographics', demographics],
    main=[ihvscatter.panel(), 
         (ihvbarchart+ profile_pipeline.hvplot.table(width=250)).panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

Please note that to get the Tabulator table styled nicely for dark mode you can set `theme='fast'` when you define the `itable`. It won't work in the notebook though.

To *serve the notebook* run `panel serve Dashboards.ipynb`.